In [1]:
#import soundfile as sf
import librosa
import numpy as np
import os
import scipy.signal as signal

In [2]:
ROOT = os.getcwd()
NUM_SAMPLES = 5
NUM_SAMPLES_PER = 10
CHUNK = 127 * 768 # 1 second = 44100 fs
SF = 44100

In [3]:
FILE_NAME = [
    '01-AchGottundHerr',
    '02-AchLiebenChristen',
    '03-ChristederdubistTagundLicht',
    '04-ChristeDuBeistand',
    '05-DieNacht',
    '06-DieSonne',
    '07-HerrGott',
    '08-FuerDeinenThron',
    '09-Jesus',
    '10-NunBitten'
    ]

train = list(np.random.choice(np.array(FILE_NAME), size=8, replace=False))
test = list()
for i in FILE_NAME:
    if i in train:
        pass
    else:
        test.append(i)

In [4]:
## Read file
def readfile(root, folder_name, file_name):
    FILE = root + '/Bach10_v1.1/' + folder_name + '/' + file_name +'.wav'
    return librosa.load(FILE, sr=SF, mono=False)

## Helper functions
def pre_process(data):
    return (data*500)+100*np.ones(shape=data.shape)

def post_process(data):
    return data#(data-100*np.ones(shape=data.shape))/500

def short_time_fourier_transform(v):
    ft = np.array(librosa.core.stft(v,n_fft=1024,hop_length=768))
    magnitude = np.abs(ft)
    phase = np.angle(ft)
    return magnitude, phase

In [5]:
def prepare_and_save_chunks(source, start_points, data, type="train"):
    output = []
    stft_mag = []
    stft_phase = []
    for i in range(len(start_points)):
        for j in start_points[i]:
            magnitude, phase = short_time_fourier_transform(data[i][j:j+CHUNK])
            output.append(data[i][j:j+CHUNK])
            stft_mag.append(magnitude)
            stft_phase.append(phase)
    # save files
    if type == "train":
        np.save(source+'_train',np.array(output))
        np.save(source+'_train_mag',np.array(stft_mag))
        np.save(source+'_train_pha',np.array(stft_phase))
    else:
        np.save(source+'_test',np.array(output))
        np.save(source+'_test_mag',np.array(stft_mag))
        np.save(source+'_test_pha',np.array(stft_phase))

    print(source)
    print("Output_Shape: {}".format(output[0].shape))
    print("Mag_Shape: {}".format(stft_mag[0].shape))
    print("Phase_Shape: {}".format(stft_phase[0].shape))
    
def generate_training_samples(names, typ):
    mix = []
    bass = []
    clar = []
    sax = []
    vio = []
    start_points = []
    for i in range(len(names)):
        # Read the file
        print('Reading file ', names[i])
        mix_data, mix_fs = readfile(ROOT, names[i], names[i])
        bass_data, bass_fs = readfile(ROOT, names[i], names[i] + '-bassoon')
        clar_data, clar_fs = readfile(ROOT, names[i], names[i] + '-clarinet')
        sax_data, sax_fs = readfile(ROOT, names[i], names[i] + '-saxphone')
        vio_data, vio_fs = readfile(ROOT, names[i], names[i] + '-violin')
        mix.append(mix_data)
        bass.append(bass_data)
        clar.append(clar_data)
        sax.append(sax_data)
        vio.append(vio_data)

        start_point = np.random.randint(0, high=bass_data.shape[0]-CHUNK, size=NUM_SAMPLES_PER)
        start_points.append(start_point)
    
    prepare_and_save_chunks('mix',start_points,mix,typ)
    prepare_and_save_chunks('bass',start_points,bass,typ)
    prepare_and_save_chunks('clar',start_points,clar,typ)
    prepare_and_save_chunks('sax',start_points,sax,typ)
    prepare_and_save_chunks('vio',start_points,vio,typ)

In [6]:
print('Generating Training Samples')

generate_training_samples(train, "train")
generate_training_samples(test, "test")

print("===== Finished Loading files =====")